this file is to find a solution by neural-network (GAN) from data (energy-base)
核心思想是energy-base的数据驱动的生成ode试探解-神经网络.
Energy-GAN for ode-trial solution
对于gan这个模型来说，我们需要的是一个生成器，一个判别器，一个自适应的损失函数
1.**第一步我们先构造一个基础的基函数的矩阵**
生成器：，他在其中进行采样，构造出一个函数f(x),系数是这个矩阵的列向量的系数。

对于判别器来说，f(x)利用在其中进行采样，在initial condition 判断这个函数是否是一个解。这里的判别器是不是二分类的问题，wgan-gp
2.**trick：对于这个基函数的矩阵 我门加一点noise，在loss下降不去的地方，说不定能试探出来**
这个case，我们就暂定是一个y=f(x)的形式,
3.**损失函数：生成器中最后泛函形式的能量，最后矩阵的变化准则：函数的内积**

矩阵形式（暂时拍脑袋）
（后期是不是可以用f-principle来选择-理论来自：NIPS 2021，
或者获取物理的先验知识，z-教师网络一起蒸馏
或者转化为含时间的决策问题-强化学习问题？
在深入需要加入图神经网络的问题吗？）
\begin{bmatrix}
 1_{t_0}& x_{t_0} &\cdots&\\
 1_{t_1}& x_{t_1} &\cdots&\\
 \vdots & \vdots  &\vdots &\\
\end{bmatrix}
4.矩阵的列是基函数的系数，行是采样点，我们可以通过这个矩阵的秩来控制基函数的个数。
**简单一点我们这里trancate列是4列，1000个点，是4*1000**（后期是不是可以随机采样？）
总结：像解算子的问题。
可优化点：1.Fenchel Conjugate共轭函数 2.互信息估计 \y_hat
～交流完看一下书，p22-23 朗斯基的导数问题





In [ ]:
import argparse
import os
import numpy as np
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch


##定义可控的生成器网络 ，输入是噪声，生成矩阵的基函数。然后在矩阵中随机采样。输出是【ini_con,t,ydot,y】。 在loss中，我们需要判断这个函数是否是一个解，用生成的“能量”来控制。
$$ w-gan的理论：lipschitz条件下的loss=D(real)-D(G(x)) $$
$$ 生成器与第一项无关，表示为G_{loss}=-D(G(x))$$
$$ 评价器，表示为D_{loss}=D(G(x))-D(real)$$
$$为满足lipschitz条件下，我们需要当输入的样本稍微变化，评价器输出的结果也要稍微变化，trick:截断参数[-1e-2,1e-2]，clip-\虽然有问题$$
为什么说是可控的呢？  因为我们的能量（基函数的选取，数值矩阵中列自己的内积）是可控的，\这对生成器的输出是有影响的,即，我们的生成器的输出是可控的，$$\可控也可以训练一个neural-network取选择列eg.x^4(\现在没有写这个功能,目前集中在生成参数\alpha_1,\alpha_2,\alpha_3,\alpha_4)$$
$$\今天的case，主要集中在loss上，如果选择列中能量很大比如x^3,我们的\alpha_4 小一点-penalty,\物理意义，节省跳跃能量，就能生成的很好$$
$$G_{loss}=-D(G(x))+\beta*energy(\alpha_1,\alpha_2,\alpha_3,\alpha_4)$$

eg. 输入的z是噪声，输出4个$$ \alpha_1,\alpha_2,\alpha_3,\alpha_4 $$

对于原来的矩阵：我们的试探函数是 $$\widetilde{\phi(t)} =\alpha_1 \psi_1(t)+...+\alpha_4 \psi_4(t) $$
*对于生成器，我们输入的是噪声，输出是\alpha_1,\alpha_2,\alpha_3,\alpha_4 在矩阵\Matrix 下的列的系数。经过组合后，生成$$y(t)（其实也可以\dot{y},这次时间比较紧，暂定y(t)，维度是【100*4】*
【ini_condition,t,yprime,\widetilde{\phi(t)}】$$


*对于判别器，是一个监督任务,$$输出是score，对第4列y(t)进行监督，由于我们有y(t)的真实值，我们需要判断这y resiual=y(t)-\widetilde{\phi(t)},
输入是【100*4】，【ini_condition,t,yprime,yresiual(t)】,输出是score

我们有y(t)的真实值，我们需要判断这个函数是否是一个解，正常来说y_resiual=0,score就越高。$$

*后期为了提高效率，是不是可以拉丁超采样或者高斯过程或者随机抽样？ 是不是loss放在判别器好一点？用能量拉开真假样本的差距 ,现在打算energy放在生成器的loss*

*使用的时候，我们输入1个随机数，能够生成类似的曲线即成功，然后拿到四个参数和基函数的解析表达式，就是有解*

*参数的非线性* eg. $$ y= \alpha_1 \psi_1(t)+...+\alpha_4 *\alpha_n\psi_4(t) $$


In [141]:
import torch
from torch.utils.data import DataLoader, Dataset,TensorDataset
#关掉warning
import warnings
import numpy as np
warnings.filterwarnings("ignore")

# 定义数据集类
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return torch.tensor(self.data[index], dtype=torch.float)

In [142]:
import sympy as sp
def create_matrix(): #数值矩阵
        # 创建符号变量
        x = sp.symbols('x')
        # 构建运算矩阵
        matrix = sp.Matrix([x**i for i in range(4)])
        # 输入 x 的值 100*1的值
        matrix_fn=sp.lambdify(x, matrix,'numpy')
        # 计算矩阵的值
        x_values = np.array([1, 2, 3, 4, 5])  # 创建一个向量
        result = matrix_fn(x_values)
        # 打印函数的符号表示
        f_symbolic = sp.pretty(matrix)
        a=result[2][0]
        print(result)
create_matrix()


[[1]
 [array([1, 2, 3, 4, 5])]
 [array([ 1,  4,  9, 16, 25])]
 [array([  1,   8,  27,  64, 125])]]


In [143]:
#把随机数定一下
import random
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
set_seed(42)

In [4]:
data=torch.load('./ode_dataset/train_data.pt')
#拿一组数据吧
data=data[0,:,:]
data=data.unsqueeze(0).float() #增加一个维度
print(data.shape)
dataset = TensorDataset(data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
for i,batch_data in enumerate(dataloader):
    print(batch_data[0])


torch.Size([1, 100, 4])
tensor([[[ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.1010,  1.0000,  0.1010],
         [ 0.0000,  0.2020,  1.0000,  0.2020],
         [ 0.0000,  0.3030,  1.0000,  0.3030],
         [ 0.0000,  0.4040,  1.0000,  0.4040],
         [ 0.0000,  0.5051,  1.0000,  0.5051],
         [ 0.0000,  0.6061,  1.0000,  0.6061],
         [ 0.0000,  0.7071,  1.0000,  0.7071],
         [ 0.0000,  0.8081,  1.0000,  0.8081],
         [ 0.0000,  0.9091,  1.0000,  0.9091],
         [ 0.0000,  1.0101,  1.0000,  1.0101],
         [ 0.0000,  1.1111,  1.0000,  1.1111],
         [ 0.0000,  1.2121,  1.0000,  1.2121],
         [ 0.0000,  1.3131,  1.0000,  1.3131],
         [ 0.0000,  1.4141,  1.0000,  1.4141],
         [ 0.0000,  1.5152,  1.0000,  1.5152],
         [ 0.0000,  1.6162,  1.0000,  1.6162],
         [ 0.0000,  1.7172,  1.0000,  1.7172],
         [ 0.0000,  1.8182,  1.0000,  1.8182],
         [ 0.0000,  1.9192,  1.0000,  1.9192],
         [ 0.0000,  2.0202,  1.0000,

In [145]:
# 定义全局变量用于记录函数被调用的次数
count_critic_step = 0
count_generator_step = 0
import matplotlib.pyplot as plt
def plot_critic_tensor_change(writer,loss,variable_t,in_fake_data:torch.Tensor,in_real_data:torch.Tensor):

    global count_critic_step
    count_critic_step+=1
    # 开启交互模式
    plt.ion()
    in_fake_data=in_fake_data.detach().numpy()
    in_fake_data=in_fake_data.reshape(100,1)
    in_real_data=in_real_data.detach().numpy()
    in_real_data=in_real_data.reshape(100,1)
    plt.figure(1)
    loss=loss.item()
    #看一个维度上最大和最小值
    plt.scatter(x=variable_t,y=in_fake_data,c='r',label='t-fake_data_critic')
    plt.scatter(x=variable_t,y=in_real_data,c='b',label='t-real_data')
    plt.legend()
    # 保存图形，并设置标题loss
    path="./tb_info/animation_csv/critic"+" "+str(count_critic_step)+".png"
    plt.title("critic_loss:"+str(loss))
    plt.savefig(path)
    #legend 图例
    plt.close()  # 清除窗口
    image = Image.open(path)
    image_tensor = ToTensor()(image)
    print(image)
    writer.add_image('critic_Image', image_tensor,global_step=count_critic_step)

def plot_generator_tensor_change(writer,loss,variable_t,in_fake_data:torch.Tensor,in_real_data:torch.Tensor):
    global count_generator_step
    count_generator_step+=1
    # 开启交互模式
    plt.ion()
    variable_t=variable_t.detach().numpy()
    variable_t=variable_t.reshape(100,1)
    in_fake_data=in_fake_data.detach().numpy()
    in_fake_data=in_fake_data.reshape(100,1)
    in_real_data=in_real_data.detach().numpy()
    in_real_data=in_real_data.reshape(100,1)
    plt.figure(1)
    loss=loss.item()

    #看一个维度上最大和最小值
    plt.scatter(x=variable_t,y=in_fake_data,c='r',label='t-fake_data_generator')
    plt.scatter(x=variable_t,y=in_real_data,c='b',label='t-real_data')
    plt.legend()
    # 保存图形，并设置标题loss
    path="./tb_info/animation_csv/genertor"+" "+str(count_generator_step)+".png"
    plt.title("generator_loss:"+str(loss))
    plt.savefig(path)
    plt.close()  # 清除窗口
    image = Image.open(path)
    image_tensor = ToTensor()(image)
    print(image)
    writer.add_image('generartor_Image', image_tensor,global_step=count_generator_step)

In [147]:

 # 定义生成器“基的系数网络”, 输入是一维度的噪声，输出是[100,1]的解
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(301, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
        )
    #kaiming 初始化
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.kaiming_normal_(m.weight,a=0,mode='fan_in')
    coeff=0 #基系数
    energy=0 #消耗的energy

    def create_matrix(self,data_t): #矩阵function

        data_numpy=data_t.clone().numpy()
        # 创建符号变量
        x = sp.symbols('x')
        # 构建运算矩阵
        matrix = sp.Matrix([x**i for i in range(4)])
        # 输入 x 的值 100*1的值
        matrix_fn=sp.lambdify([x], matrix,'numpy')
        # 计算矩阵的值
        result = matrix_fn(data_numpy)
        # 打印函数的符号表示
        f_symbolic = sp.pretty(matrix)
        return result[1][0],result[2][0],result[3][0]#返回三个列，表示生成的列，这里数值为1的列没有返回，后面有*vector，后期待优化

    def print_coeff(self):
        print('coeff is',self.coeff)
        return self.coeff

    def calculate_generate(self,coeff,data_t): #计算生成的函数
        #print(coeff)
        # 创建符号变量
        #构建100*1的numpy
        num_matirx=np.ones((4,100,1))
        num_matirx[1,:,:],num_matirx[2,:,:],num_matirx[3,:,:]=self.create_matrix(data_t)
        #print(num_matirx.shape)
        num_matirx_tensor=num_matirx.astype(np.float32)
        num_matirx_tensor=torch.from_numpy(num_matirx)
        #四列加起来成为一列
        generate_data=coeff[0,0]*data_t \
                      #+coeff[0,1]*num_matirx_tensor[1,:,:]\
                      # +0*coeff[0,2]*num_matirx_tensor[2,:,:]\
                      # +0*coeff[0,3]*num_matirx_tensor[3,:,:]
        #data_t用二范数
        # 计算一列数据的二范数 作为能量
        energy=torch.norm(data_t)\
               +torch.norm(num_matirx_tensor[1,:,:])\
               +torch.norm(num_matirx_tensor[2,:,:])\
               +torch.norm(num_matirx_tensor[3,:,:])
        return generate_data

    def forward(self, x,data_t):
        self.coeff=self.model(x)

        data=self.calculate_generate(self.coeff,data_t)
        return data

# 定义判别器网络 输入是【ini_con,t,ydot,y】，输出是一个数-打分
#[100,4]
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 100),
        )
         #kaiming 初始化
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.kaiming_normal_(m.weight,a=0,mode='fan_in')

    def forward(self, x):
        print(x.shape)
        return self.model(x)

In [148]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb
from PIL import Image
from torchvision.transforms import ToTensor
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
# 创建一个 SummaryWriter 对象，指定保存日志的目录
writer = SummaryWriter('./tb_info/logs')
# 初始化生成器和判别器
generator = Generator()
discriminator = Discriminator()

# 优化器,  固定学习率1e-3
optimizer_g = optim.Adam(generator.parameters(), lr=1e-4)
optimizer_d = optim.Adam(discriminator.parameters(), lr=1e-4)

# 获取一个批次的数据
batch_data = next(iter(dataloader))
# 训练生成器每n_generator步
n_generator=100
# 训练生成器每n_critic步
n_critic=1
beta=0
# 训练GAN模型
num_epochs = 10000
noise_dim = 1  # 噪声向量的维度
mean = 0.0  # 高斯分布的均值
stddev = 0.01   # 高斯分布的标准差
Dict_G={"train_critic_fake_y":torch.tensor([]),
        "train_self":torch.tensor([])}

for epoch in range(num_epochs):
    for i,batch_data in enumerate(dataloader,0):#索引值将从 0 开始递增，依次对应每个批次的位置。
        # 训练判别器
        real_data=batch_data[0].float()
        real_y_data=real_data[:,:,3]
        condition_data=real_data[:,:,0:3].float()
        condition_data=condition_data.reshape(-1,300)
        print("real_y_data_shape",real_y_data.shape)#【1,100,4】
        # 定义噪声向量的大小和分布参数
        z = torch.randn((1,noise_dim))*stddev+mean#1*noise_dim
        z_condition=torch.cat((z,condition_data),dim=1)#1*（noise_dim+300）
        #这里也记录一下采样的噪声值
        writer.add_scalar('train_critic_z', z, epoch)
        real_data_t=real_data[:,:,1]#变量t
        real_data_t=real_data_t.reshape(-1,1)
        #生成假的y数据 要detach
        fake_y_data = generator(z_condition.detach(),real_data_t.detach()).float()
        print("fake_y_data*",fake_y_data.shape)
        fake_y_data=fake_y_data.view(1,100)
        print("fake_y_data*",fake_y_data.shape)
        print("real_y_data*",real_y_data.shape)
        #压缩成一列

        #real_scores = discriminator(real_y_data)
        fake_scores = discriminator(fake_y_data)

        # 计算判别器损失
        #d_loss =fake_scores-real_scores#神经网络打分器

        d_loss=-torch.nn.MSELoss()(fake_scores,real_y_data)#神经网络打分器
        print(d_loss)
        plot_critic_tensor_change(writer,d_loss,real_data_t,fake_y_data,real_y_data)

        # 反向传播和更新判别器的参数
        d_loss.backward()
        optimizer_d.step()
        #梯度截断
        for p in discriminator.parameters():
            p.data.clamp_(-0.01, 0.01)
    writer.add_scalars('losses', {'critic':d_loss.item()}
                                         , epoch)#记录

        # 训练生成器
        #取余
    if((epoch+1)%n_generator)==0:#训练生成器
            # -----------------
            #  训练生成器
            # -----------------
            optimizer_g.zero_grad()

            data_t=real_data[:,:,1].detach()#变量t
            condition_data=real_data[:,:,0:3].float()
            condition_data=condition_data.reshape(-1,300)
            data_t=data_t.reshape(-1,1) #[100,1]

            supervisor_y=real_data[:,:,3]#
            supervisor_y=supervisor_y.reshape(100,1)

            # 生成噪声作为生成器输入
            z = torch.randn((1,noise_dim))*stddev+mean#1*noise_dim#噪声
            z_condition=torch.cat((z,condition_data),dim=1)#1*（noise_dim+300）
            # 生成器输出
            fake_y_data = generator(z_condition.detach(),data_t.detach())
            fake_y_data=fake_y_data
            fake_y_data=fake_y_data.view(1,100)
            print("fake_y_data shape",fake_y_data.shape)

            supervisor_y=supervisor_y.reshape(100,1)

            g_loss = -torch.mean(discriminator(fake_y_data.float()))
            print("gloss_梯度地址",g_loss)
            plot_generator_tensor_change(writer,g_loss,data_t,fake_y_data,supervisor_y)


            writer.add_scalars('losses', {'generator':g_loss.item()}
                                         , epoch)#记录
            #记录一下 训练生成器的系数
            coeff=generator.print_coeff()
            writer.add_scalars('trian_generator_coeff',
                                     {'alpha1':coeff[0,0].item(),
                                     #'alpha2':coeff[0,1].item(),
                                     # 'alpha3':coeff[0,2].item(),
                                      #'alpha4':coeff[0,3].item()
                                      }
                                         , epoch)#记录

            #反向传播
            g_loss.backward()
            optimizer_g.step()

    # 关闭 SummaryWriter
    writer.close()
 #每个epoch结束后打印损失
    #print(f"Epoch [{epoch+1}/{num_epochs}], Generator Loss: {g_loss.item():.4f}, Discriminator Loss: {d_loss.item():.4f}")

real_y_data_shape torch.Size([1, 100])
fake_y_data* torch.Size([100, 1])
fake_y_data* torch.Size([1, 100])
real_y_data* torch.Size([1, 100])
torch.Size([1, 100])
tensor(-238.6468, grad_fn=<NegBackward0>)
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=640x480 at 0x60BCF9F30>
real_y_data_shape torch.Size([1, 100])
fake_y_data* torch.Size([100, 1])
fake_y_data* torch.Size([1, 100])
real_y_data* torch.Size([1, 100])
torch.Size([1, 100])
tensor(-33.4998, grad_fn=<NegBackward0>)
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=640x480 at 0x2CFFF5E10>
real_y_data_shape torch.Size([1, 100])
fake_y_data* torch.Size([100, 1])
fake_y_data* torch.Size([1, 100])
real_y_data* torch.Size([1, 100])
torch.Size([1, 100])
tensor(-33.5009, grad_fn=<NegBackward0>)
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=640x480 at 0x6C886C040>
real_y_data_shape torch.Size([1, 100])
fake_y_data* torch.Size([100, 1])
fake_y_data* torch.Size([1, 100])
real_y_data* torch.Size([1, 100])
torch.Size([1, 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# 输入噪声
z = torch.randn((1,noise_dim))*stddev+mean#1*noise_dim
print(z)
data_t=torch.linspace(0,10,100)

data_t=data_t.reshape(-1,1)
fake_data = generator(z,data_t)
generator.print_coeff()
a=fake_data.detach().numpy()
# 画出生成器生成的数据分布
plt.plot(data_t,a[:,0],label='G(t)')
plt.plot(data_t,data[0,:,3],label='raw(t)')
plt.legend()


In [127]:
import torch

# 定义张量的大小
rows = 1
cols = 2

# 定义噪声的均值和方差
mean = 0
variance = 4
std_dev = torch.sqrt(torch.tensor([variance]))  # 标准差是方差的平方根

# 使用 PyTorch 生成二维的噪声
noise = torch.randn(rows, cols) * std_dev + mean  # 将噪声调整为具有所需的均值和方差

print(noise)

tensor([[0.5519, 3.1675]])


tensor([[ 3.7067, -1.5636]])
